In [1]:
pip install cri==1.8.1

     |████████████████████████████████| 2.4MB 1.6MB/s eta 0:00:01
  Created wheel for cri: filename=cri-1.8.1-cp37-none-any.whl size=30859 sha256=d593b48a8067217845d37fd1077c21a1f0bcb148362ed3ef4524c7f078a60421
  Stored in directory: /home/jovyan/.cache/pip/wheels/f4/8a/a1/47e4f97d695099a9fcb1cb6dfe7fcb42ed1a9536114f55eead
Successfully built cri
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install 'joblib==0.11'

     |████████████████████████████████| 184kB 1.0MB/s eta 0:00:01
  Found existing installation: joblib 0.13.2
    Uninstalling joblib-0.13.2:
      Successfully uninstalled joblib-0.13.2
Note: you may need to restart the kernel to use updated packages.


In [3]:
import grpc
from cri import api_pb2

channel = grpc.insecure_channel('unix:///var/run/docker.sock')
runtime_stub = api_pb2.RuntimeServiceStub(channel)
2 = api_pb2.ImageServiceStub(channel)


sandboxConfig = api_pb2.PodSandboxConfig(
    metadata=api_pb2.PodSandboxMetadata(name="sandbox", namespace="test"),
    dns_config=api_pb2.DNSConfig(servers=["3.3.3.3"], searches=["google.com"])
)

sandbox_resp = runtime_stub.RunPodSandbox(
    api_pb2.RunPodSandboxRequest(config=sandboxConfig))

print image_stub.PullImage(api_pb2.PullImageRequest(image=api_pb2.ImageSpec(image="busybox")))

containerConfig = api_pb2.ContainerConfig(
    metadata=api_pb2.ContainerMetadata(name="busybox"),
    image=api_pb2.ImageSpec(image="busybox"),
    command=["sh", "-c", "top"],
)

container_resp = runtime_stub.CreateContainer(api_pb2.CreateContainerRequest(
    pod_sandbox_id=sandbox_resp.pod_sandbox_id,
    config=containerConfig,
    sandbox_config=sandboxConfig,
))

print runtime_stub.StartContainer(api_pb2.StartContainerRequest(
    container_id=container_resp.container_id))

SyntaxError: invalid syntax (<ipython-input-3-da85b1e76f65>, line 17)

In [ ]:
channel = grpc.insecure_channel('unix:///var/run/dockershim.sock')
runtime_stub = api_pb2.RuntimeServiceStub(channel)
image_stub = api_pb2.ImageServiceStub(channel)

# Get runtime version
print runtime_stub.Version(api_pb2.VersionRequest())

# Get sandboxes
print runtime_stub.ListPodSandbox(api_pb2.ListPodSandboxRequest())

# Get Images
print image_stub.ListImages(api_pb2.ListImagesRequest())

In [5]:
!pip install docker

     |████████████████████████████████| 143kB 1.1MB/s eta 0:00:01


In [6]:
import docker

In [7]:
client = docker.from_env()

In [8]:
client

In [9]:
client.containers.run("ubuntu", "echo hello world")

ConnectionError: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory'))

In [22]:
client.containers.list()

[<Container: a13541b76f>,
 <Container: 7d0b9db2e6>,
 <Container: a25a907ebd>,
 <Container: 2143b3c28d>,
 <Container: 2343d448b9>,
 <Container: 6c48ecf498>,
 <Container: d4599b939b>]

In [23]:
client.images

In [24]:
client.images.build

<bound method ImageCollection.build of <docker.models.images.ImageCollection object at 0x7f15b9f670f0>>

In [25]:
mkdir docker

mkdir: cannot create directory ‘docker’: File exists


In [26]:
cd docker

/home/michele/Projects/rmi-bambino-gesu/docker


In [27]:
ls

freesurfer/  mcflirt/


In [28]:
cd ..

/home/michele/Projects/rmi-bambino-gesu


In [29]:
mkdir test

In [30]:
cd test

/home/michele/Projects/rmi-bambino-gesu/test


In [45]:
content = """
FROM python:3.6-stretch
MAINTAINER Michele Mastrogiovanni <michele.mastrogiovanni@gmail.com>

# install build utilities
RUN apt-get update && \
	apt-get install -y gcc make apt-transport-https ca-certificates build-essential

# check our python environment
RUN python3 --version
RUN pip3 --version

# set the working directory for containers
WORKDIR  /usr/src/<app-name>

# Installing python dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy all the files from the project’s root to the working directory
COPY src/ /src/
RUN ls -la /src/*

# Running Python Application
CMD ["python3", "/src/main.py"]
"""

f = open("Dockerfile","w+")
f.write(content)
f.close()

f = open("requirements.txt", "w+")
f.close()

import os
os.mkdir("src")

f = open("src/main.py", "w+")
content = """
print("Hello Stocazzo")
"""
f.write(content)
f.close()

In [53]:
ls *

Dockerfile  Dockerfile.txt  requirements.txt

src:
main.py


In [54]:
(image, logs) = client.images.build(path="./", tag="stocazzo")

In [56]:
for log in logs:
    if log['stream']:
        print(log['stream'], end = '')
    print(log)

Successfully built e9ba4af556dc
{'stream': 'Successfully built e9ba4af556dc\n'}
Successfully tagged stocazzo:latest
{'stream': 'Successfully tagged stocazzo:latest\n'}


In [57]:
client.containers.run("stocazzo")

b'Hello Stocazzo\n'